## Overlaps Experiments
- This example uses the underlying gpd R-tree to calculate intersections between polygons.

In [17]:
from datetime import datetime
import logging
import os

import geopandas as gpd
import matplotlib as plt
from python_gis.poc.landgrid_processor import LandgridProcessor
from python_gis.poc.io.pgsql import PgWriter

import python_gis.poc.util.log_config

logger = logging.getLogger(__name__)

In [ ]:
!ls $DIML_HOME

In [2]:
# Postgis
pg_host = os.environ["PG_SERVER"]
pg_db = os.environ["PG_DATABASE"]
pg_user = os.environ["PG_UID"]
pg_pwd = os.environ["PG_PWD"]

# WGS84 (epsg:4326)
gdb_path = os.path.join(os.environ["DATA_DIR"], 'landgrid', 'DI_basemaps_WGS84.gdb')
ddl_path = os.path.join(os.environ["DIML_HOME"], 'database', 'schema.sql')
out_path = os.path.join(os.environ["DATA_DIR"], 'shapefile_out')
idx_path = os.path.join(os.environ["DIML_HOME"], 'database', "indexes.sql")

### Read files and create sindexes

In [13]:
county_path = os.path.join(out_path, 'Counties_US_WGS84_clean.shp')
county_df = gpd.read_file(county_path)
county_idx = county_df.sindex

In [14]:
child_path = os.path.join(out_path, f'AZ_section_clean.shp')
az_sec_df = gpd.read_file(child_path).loc[:10000, :]
az_idx = az_sec_df.sindex

In [5]:
az_sec_df.head(1)

,StateID,StateAPI,TWPCODE,SECCODE,MER,MST,TWP,THALF,TNS,RGE,RHALF,REW,SEC,Shape_Leng,Shape_Area,geometry
0,61,02,0214T00300NR02300E,0214T00300NR02300E24,14,Gila and Salt River,3,0,N,23,0,E,24,0.063664,0.000251,"POLYGON ((-109.90901 33.59656, -109.89170 33.5..."


### Run spatial join to get baseline results

In [15]:
olaps_df = gpd.sjoin(az_sec_df, county_df, how='inner', op='intersects')

In [22]:
olaps_df.loc[0, ['SECCODE', 'County_Nam']]

,SECCODE,County_Nam
0,0214T00300NR02300E24,Apache
0,0214T00300NR02300E24,Navajo


### Pick a random county and section

In [20]:
county_polygon = county_df.loc[0, :]

County_Nam                                              Autauga
State_Name                                              Alabama
CountyID                                                    577
StateID                                                      60
FIPS_State                                                   01
FIPS_Count                                                  001
API_State                                                    01
API_County                                                  001
LAT                                                      32.535
LON                                                    -86.6428
Shape_Leng                                              2.06526
Shape_Area                                              0.15026
geometry      POLYGON ((-86.41313722699994 32.70749287800004...
Name: 0, dtype: object

In [24]:
section_polygon = az_sec_df.loc[0, :].geometry

### Get the intersection of counties with the section's bounding box

In [25]:
county_overlaps = list(county_idx.intersection(section_polygon.bounds))
county_overlaps

[1799, 1795, 1804]

In [27]:
possible_matches = county_df.iloc[county_overlaps]
possible_matches

,County_Nam,State_Name,CountyID,StateID,FIPS_State,FIPS_Count,API_State,API_County,LAT,LON,Shape_Leng,Shape_Area,geometry
1799,Graham,Arizona,651.0,61.0,04,009,02,009,32.932825,-109.887475,5.462724,1.158908,"POLYGON ((-109.49546 33.65279, -109.49546 33.6..."
1795,Apache,Arizona,647.0,61.0,04,001,02,001,35.395687,-109.488869,9.317715,2.883117,"POLYGON ((-109.04524 36.99916, -109.04524 36.9..."
1804,Navajo,Arizona,656.0,61.0,04,017,02,017,35.399711,-110.321430,8.747960,2.559704,"POLYGON ((-110.00070 36.99803, -110.00070 36.9..."


### Use the possible county matches and the original polygon to get the actual intersection
- This should match the results from the sjoin earlier

In [28]:
actual_matches = possible_matches[possible_matches.intersects(section_polygon)]
actual_matches

,County_Nam,State_Name,CountyID,StateID,FIPS_State,FIPS_Count,API_State,API_County,LAT,LON,Shape_Leng,Shape_Area,geometry
1795,Apache,Arizona,647.0,61.0,04,001,02,001,35.395687,-109.488869,9.317715,2.883117,"POLYGON ((-109.04524 36.99916, -109.04524 36.9..."
1804,Navajo,Arizona,656.0,61.0,04,017,02,017,35.399711,-110.321430,8.747960,2.559704,"POLYGON ((-110.00070 36.99803, -110.00070 36.9..."
